In [1]:
import sys
from pathlib import Path
import os

os.chdir("/workspaces/Gini-index-prediction")
print("Current working directory:", os.getcwd())


project_root = Path().resolve()
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))
    print("src added to sys.path:", src_path)
else:
    print("src already in sys.path:", src_path)


import pandas as pd

from src.preprocessing import clean_inequality_data
from src.split_data import split_data
from src.models import compare_models


df = clean_inequality_data()
df.head()

X_train, X_test, y_train, y_test = split_data(df, "gini")


baseline_results = compare_models(X_train, X_test, y_train, y_test)
baseline_results




df_inter = df.copy()

df_inter["female_x_education"] = (
    df_inter["female_labor_participation"] * df_inter["education_secondary_enrollment"]
)
df_inter["urban_x_gdp"] = df_inter["urbanization"] * df_inter["gdp_pc"]
df_inter["female_x_gdp"] = df_inter["female_labor_participation"] * df_inter["gdp_pc"]
df_inter["trade_x_growth"] = df_inter["trade_openness"] * df_inter["gdp_growth"]

df_inter.head()


X_train_int, X_test_int, y_train_int, y_test_int = split_data(df_inter, "gini")


inter_results = compare_models(X_train_int, X_test_int, y_train_int, y_test_int)
inter_results


comparison = baseline_results.merge(
    inter_results,
    on="model",
    suffixes=("_baseline", "_with_interactions")
)

comparison





Current working directory: /workspaces/Gini-index-prediction
src added to sys.path: /workspaces/Gini-index-prediction/src


Loading raw data from: data/wb_inequality_data.xlsx
Initial shape: (9044, 13)
Number of countries with ≥ 10 GINI values: 78
After country filter: (2652, 13)
After year filter (1995-2020): (2028, 13)
After dropping rows without Gini: (1528, 13)
Numeric columns to interpolate: ['gdp_pc', 'gdp_growth', 'population', 'female_labor_participation', 'labor_participation', 'education_secondary_enrollment', 'urbanization', 'trade_openness', 'tax_revenue']

 Number of NA per column after cleaning:
country                           0
country_code                      0
year                              0
gini                              0
gdp_pc                            0
gdp_growth                        0
population                        0
female_labor_participation        0
labor_participation               0
education_secondary_enrollment    0
urbanization                      0
trade_openness                    0
tax_revenue                       0
dtype: int64

Cleaned data exported to:

,model,r2_baseline,rmse_baseline,mae_baseline,r2_with_interactions,rmse_with_interactions,mae_with_interactions
0,Linear Regression,0.6066,5.4935,4.3785,0.6245,5.3671,4.3329
1,Random Forest,0.9287,2.3381,1.6117,0.9256,2.3896,1.6598
2,XGBoost,0.9383,2.1749,1.6435,0.9359,2.2180,1.6644
